In [40]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import matplotlib.pyplot as plt
import matplotlib.font_manager as fm


from codes.appendix import SectCode
from codes.appendix import FatrCode

In [41]:
from IPython.core.display import HTML

# 글씨 크기를 10px로 조절하는 CSS
css = """
<style>
table.dataframe td, table.dataframe th {
    font-size: 10px;
}
</style>
"""
HTML(css)

In [42]:
from src.api import SmartFarmAPI

api = SmartFarmAPI(config="config.yaml")

SerlNo(작기 일련번호)를 ID로 삼고, 사용자가 원하는 작기 ID를 merge하는 클래스를 제작하였습니다.

In [ ]:
class TomatoDataProcessor:
    def __init__(self, serl_nos):
        self.serl_nos = serl_nos
        self.all_page_data = []


    def get_data_for_page(self, serl_no,page):
        return pd.DataFrame(api.crop_season.getCroppingSeasonEnvDataList(serl_no, page))
        
        
    def get_data_all_pages(self, serl_no):
        first_page_data=self.get_data_for_page(serl_no,1)
        total_page = int(float(first_page_data.at[0, 'totalPage']))

        all_data=[]
        for page in range(1, total_page+2):
            page_data = self.get_data_for_page(serl_no, page)
            page_data['SerlNo'] = serl_no
            all_data.append(page_data)
        return pd.concat(all_data)

    def transform_data(self, final_df):
        df_selected = final_df[["measDate", "fldCode", "sectCode", "fatrCode", "senVal","SerlNo"]]
        df_selected["senVal"] =  pd.to_numeric(df_selected["senVal"], errors='coerce')
        df_selected["measDate"] = pd.to_datetime(df_selected["measDate"])
        df_pivoted = df_selected.pivot_table(index=['measDate', 'SerlNo'], columns='fatrCode', values='senVal', aggfunc='mean')
        df_pivoted.rename(columns=lambda x: FatrCode.get_description(x), inplace=True)
        return df_pivoted

    def run(self):
        for no in self.serl_nos:
            all_page_data = self.get_data_all_pages(no)
            self.all_page_data.append(all_page_data)

        final_df = pd.concat(self.all_page_data, ignore_index=True)
        transformed_df = self.transform_data(final_df)
        return transformed_df

if __name__ == "__main__":
    df = pd.DataFrame(api.crop_season.getCroppingSeasonDataList("2021"))
    df_tomato = df[(df["itemName"] == "토마토") & (df["acqAutoYn"] == "Y")]
    df_tomato['croppingSerlNo'].unique()
    serl = np.array([4419,4420,4430])
    processor = TomatoDataProcessor(serl)
    '''
    임의로 3곳의 serl을 입력하였지만, df_tomato에는 더 많은 작기가 있습니다. 이런 식으로 np.array로 serl을 입력하면, 해당 serl에 대한 데이터를 가져올 수 있습니다.
    '''
    df_pivoted = processor.run()
    print(df_pivoted)
 


In [133]:
df_pivoted

,measDate,fldCode,sectCode,fatrCode,senVal,SerlNo
0,2021-01-12 00:00:00,FG,EI,CI,606.17,4419
1,2021-01-12 00:00:00,FG,EI,HI,92.77,4419
2,2021-01-12 00:00:00,FG,EI,TI,15.33,4419
3,2021-01-12 00:00:00,FG,EO,TE,-3.91,4419
4,2021-01-12 01:00:00,FG,EI,HI,95.14,4419
...,...,...,...,...,...,...
99847,2021-07-04 23:00:00,FG,EI,TI,15.48,4430
99848,2021-07-04 23:00:00,FG,EL,TL,24.18,4430
99849,2021-07-04 23:00:00,FG,EO,RP,0.25,4430
99850,2021-07-04 23:00:00,FG,EO,TE,24.00,4430
